# Boson Sampling

In [9]:
import numpy as np
import piquasso as pq


with pq.Program() as program:
    pq.Q() | pq.SamplingState(0, 1, 1, 0)

    pq.Q(0, 1) | pq.Beamsplitter(theta=np.pi / 3, phi=np.pi / 4)

    pq.Q(2, 3) | pq.Beamsplitter(theta=np.pi / 2, phi=np.pi / 3)

    pq.Q() | pq.Sampling(shots=5)

results = program.execute()

print(results)

[<Result instruction=<pq.Sampling(shots=5, modes=(0, 1, 2, 3))> samples=[array([1, 0, 0, 1]), array([0, 1, 0, 1]), array([1, 0, 0, 1]), array([0, 1, 0, 1]), array([1, 0, 0, 1])]>]
